# Use NumPy to recognize handwritten digits

In [2]:
from PIL import Image
import numpy as np

Download the data. In this case, MNIST dataset.

In [3]:
import os
from urllib.request import urlretrieve
dataset = 'mnist.pkl.gz'
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(min(a*b*100.0/c,100)), end="")
base_url = 'https://github.com/mrgloom/MNIST-dataset-in-different-formats/raw/master/data/Original%20dataset/'
dataset_files = ['t10k-images.idx3-ubyte', 't10k-labels.idx1-ubyte', 'train-images.idx3-ubyte', 'train-labels.idx1-ubyte']    
for file in dataset_files:
    if not os.path.isfile(file):
        origin = base_url + file
        print('Downloading data %s' % file)
        urlretrieve(origin, file, reporthook=reporthook)
        print('\n')

In [4]:
!dir *.idx1-ubyte

 Volume in drive C is Windows
 Volume Serial Number is E6A5-2830

 Directory of c:\Users\User\src\DataScienceSoftware2024\02-machine_learning_part_1_2

09/17/2024  12:35 PM            10,008 t10k-labels.idx1-ubyte
09/17/2024  12:35 PM            60,008 train-labels.idx1-ubyte
               2 File(s)         70,016 bytes
               0 Dir(s)  1,295,095,435,264 bytes free


# THE IDX FILE FORMAT
the IDX file format is a simple format for vectors and multidimensional matrices of various numerical types.

The basic format is
```
magic number
size in dimension 0
size in dimension 1
size in dimension 2
.....
size in dimension N
data
```

The magic number is an integer (MSB first). The first 2 bytes are always 0.

```
The third byte codes the type of the data:
0x08: unsigned byte
0x09: signed byte
0x0B: short (2 bytes)
0x0C: int (4 bytes)
0x0D: float (4 bytes)
0x0E: double (8 bytes)
```


The 4-th byte codes the number of dimensions of the vector/matrix: 1 for vectors, 2 for matrices....


The sizes in each dimension are 4-byte integers (MSB first, high endian, like in most non-Intel processors).


The data is stored like in a C array, i.e. the index in the last dimension changes the fastest.

In [5]:
for file in dataset_files:
    magic = np.fromfile(file, dtype='uint8', offset=0, count=4)
    print(file, magic)
    _, _, dtype, n_dim = magic
    assert dtype == 8
    assert n_dim < 4
    shape = np.fromfile(file, dtype='>i4', offset=4, count=n_dim)
    print('shape', [x for x in shape])

t10k-images.idx3-ubyte [0 0 8 3]
shape [10000, 28, 28]
t10k-labels.idx1-ubyte [0 0 8 1]
shape [10000]
train-images.idx3-ubyte [0 0 8 3]
shape [60000, 28, 28]
train-labels.idx1-ubyte [0 0 8 1]
shape [60000]


In [6]:
def read_dataset(prefix):
    images = np.fromfile(prefix + '-images.idx3-ubyte', dtype='uint8', offset=16).reshape(-1, 28*28)/255
    labels = np.fromfile(prefix + '-labels.idx1-ubyte', dtype='uint8', offset=8)
    return images, labels
train_set = read_dataset('train')
test_set = read_dataset('t10k')

## Q
First, check shape and dtype of the dataset.

In [7]:
train_set

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8))

In [8]:
%run -i q_see_mnist_data.py

train_set[0]:      shape=(60000, 784)	 dtype=float64
train_set[1]:      shape=(60000,)	 dtype=uint8


NameError: name 'validation_set' is not defined

# Supervised Learning

<img src="supervised.svg" />
vs
<img src="supervised2.svg" />
See also Chinese room

In [9]:
train_X, train_y = train_set
test_X, test_y = test_set

### Look into the content. What is in `y`?

In [10]:
train_y[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [11]:
train_X.shape

(60000, 784)

In [12]:
# Let's check the first 20 items in the training data
train_y[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

### What is in `X`?

In [13]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# show the first 20 samples in training X
showX(train_X[120:150])

In [14]:
showX(test_X[:10])

In [15]:
test_y.shape

(10000,)

# nearest neighbor

### Let's start from the simplest mehods

Always start from the obvious, simple, boring methods, 

For instance, in order to decide the digit in a handwritten sample, can we simply identify the most similar image in the training data, and assume they represents the same digit?

It may appear naive, but why not give it a shot?

But what exactly does similarity mean? You know, it is kind of tricky to define the similarity of two images. Some approaches, such as SSIM are related to human visual neurobiology and perception, and some methods like LPIPS employ neural networks. 

Again, always begin with the obvious, simple, boring methods. Let us using sqauare error to measure the differences between two images.

## Q 
How to calculate the square difference between two vectors u and v?
```python
u = train_X[0]
v = train_X[1]
```

Note that the L2 norm of a vector $u=(x,y)$ is $\left\Vert u \right\Vert=\sqrt{x^2+y^2}$.


In [16]:
showX(train_X[:2])

In [17]:
# %load q_square_error.py
u = train_X[0]
v = train_X[1]
print ( ((u - v)**2).sum() )
# or using np.linalg.norm to calculate the Euclidean distance, i.e. L2 distance
print ( np.linalg.norm(u-v)**2 )

87.63247981545558
87.6324798154556


In [18]:
showX(train_X[0:2])

In [19]:
# you may also use the following code to calculate the absolute error or the L1 distance
print(np.abs(test_X[0] - train_X[1]).sum())
# or 
print(np.linalg.norm(test_X[0] - train_X[1], ord=1))

135.50196078431372
135.50196078431372


In [20]:
showX(test_X[1:2])

The following celll essentially finds and displays the training sample that is closest to the given test sample based on the Euclidean distance.

In [21]:
target_id = 3217 # an arbitrary test sample id
X = test_X[target_id]
print("target:", test_y[target_id])
showX(X)
# calculate the distance between X and each training sample
best_dst, best_i = 100000, -1
for i in range(60000):
    dst = ((X - train_X[i])**2).sum()
    if dst < best_dst:
        best_dst, best_i = dst, i
        print(i, dst, train_y[i])
        showX(np.array([X, train_X[i]] ))

target: 3


0 115.64461361014995 5


3 115.48529027297194 1


6 101.90505190311418 1


8 100.6317570165321 1


...

3596 33.00687427912341 3


25282 31.47277201076509 3


33444 30.057116493656288 3


35422 28.60413687043445 3


48725 21.68461361014994 3


In [22]:
showX(train_X[best_i])
print("best match:", train_y[best_i])

best match: 3


## Q 
### Using the numpy way to find the nearest neighbor.
i.e. try not to use the `for` loop 

* Show test_X[0]
* Find a sample in  `train_X` which is most similar to `test_X[0]`, without using a loop.
* display the image of that sample
* check the corresponding  `train_y`
* see if it is the same as test_y[0]

In [23]:
# %load q_find_nn_0.py
# display test_X[0]
showX(test_X[0])
# claculate the distance between test_X[0] and each training sample
_ = ((train_X - test_X[0])**2).sum(axis=1)
# find the index of the sample that has the smallest distance
idx = _.argmin()
print("train_X[{}]".format(idx))
showX(train_X[idx])
print("train_X[{}] = {}".format(idx, train_y[idx]))
print("test_y[0] =", test_y[0])


train_X[53843]


train_X[53843] = 7
test_y[0] = 7


## Q

Take the first 10, 100 samples from `test_X` and do the same. What is the accuracy of this method?

In [24]:
# %load q_find_nn_10.py
from tqdm.auto import tqdm # for showing the progress bar
N = 100
correct = 0
for i in tqdm(range(N)):
    # display test_X[i]
    #if i%30==0:
    #    print("test_X[{}]".format(i))
    #showX(test_X[i])
    # calculate the distance between test_X[i] and each training sample
    distances = ((train_X - test_X[i])**2).sum(axis=1)
    # find the index of the sample that has the smallest distance
    idx = distances.argmin()
    #print("train_X[{}]".format(idx))
    #showX(train_X[idx])
    #print("train_X[{}] = {}".format(idx, train_y[idx]))
    #print("train_X[{}] = {}".format(i, test_y[i]))
    if train_y[idx] == test_y[i]:
        correct+=1
print("Accuracy", correct/N)


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy 1.0


## Q
What if train_X cotains only 500 samples?

use reshaping,  broadcasting to caculate the accuracy for `test_X[:100]`.

Hint: Can use `np.expand_dims` instead of `np.reshape`.

In [25]:
# !may use too much memory
#%run -i q_small_data.py
# accuracy: 85%

## Get rid of the loop?

In [28]:
U = train_X.reshape((-1, 1, 784))
U.shape

(60000, 1, 784)

In [29]:
V = test_X[:30].reshape((1, -30, 784))
V.shape

(1, 30, 784)

In [30]:
R = ((U-V)**2) .sum(axis=2)
R.shape

(60000, 30)

In [31]:
# however, the above code may use too much memory
# even with 100 samples, the memory usage is about 18GB
# let alone 10000 samples
K = 1024
M = 1024*K
G = 1024*M
(60000 * 100 * 784* 4)/G

17.523765563964844

In [32]:
(U-V).shape

(60000, 30, 784)

In [37]:
(train_y[R.argmin(axis=0)] == test_y[:30]).mean()

1.0

In [38]:
test_y[:30]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1], dtype=uint8)

## Q
What about using other distance functions? e.g.  `np.abs(...).sum()`

### Use inner product instead of square difference
$$
\begin{align*}
\left\Vert \mathbf{u}-\mathbf{v}\right\Vert ^{2} & =\left(\mathbf{u}-\mathbf{v}\right)\cdot\left(\mathbf{u}-\mathbf{v}\right)\\
 & =\left\Vert \mathbf{u}\right\Vert ^{2}-2\mathbf{u}\cdot\mathbf{v}+\left\Vert \mathbf{v}\right\Vert ^{2}\\
\end{align*}
$$


In [39]:
# make a copy of the original data
train_X0 = train_X
test_X0 = test_X

In [40]:
# normalize the data, so that the cosine similarity is equivalent to the dot product
train_X  = train_X0 / np.linalg.norm(train_X, axis=1, keepdims=True)
test_X  = test_X0 / np.linalg.norm(test_X, axis=1, keepdims=True)

In [41]:
# matrix multiplication == pairwise computation of inner product
A = test_X @ train_X.T
print(A.shape)

(10000, 60000)


In [43]:
A.argmax(axis=1)

array([53843, 28882, 58741, ...,  3261,  1311, 22424], dtype=int64)

In [44]:
predict_y = train_y[A.argmax(axis=1)]
predict_y

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [45]:
# Show first 20 samples of  Test X
showX(test_X0[:20])

In [46]:
# predicted Y
predict_y[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=uint8)

In [47]:
# First 20 samples of Test y
test_y[:20]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4],
      dtype=uint8)

In [48]:
# accuracy
(predict_y == test_y).mean()

0.9723

### Employ PCA to reduce dimensions
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


In [46]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)
train_X = pca.fit_transform(train_set[0])
test_X = pca.transform(test_set[0])

In [47]:
train_X.shape

(50000, 100)

In [31]:
np.mean((train_set[0][0]-train_set[0][1])**2)

0.110904574

In [32]:
np.mean((train_X[0]-train_X[1])**2)

0.11090599

In [33]:
m = train_set[0].mean(axis=0)
m.shape

(784,)

In [34]:
train_X[0]@train_X[1]

10.671793

In [35]:
(train_set[0][0]-m)@(train_set[0][1]-m)

10.671726

In [36]:
train_X /= np.linalg.norm(train_X, axis=1, keepdims=True)
test_X /= np.linalg.norm(test_X, axis=1, keepdims=True)

In [37]:
# using matrix multiplication
A = test_X @ train_X.T

In [38]:
predict_y = train_y[A.argmax(axis=1)]

In [39]:
# accuracy
(predict_y == test_y).mean()

0.9669

## Q
* Try other settings of PCA
* Check the differences
* use sklearn knn